# Prompt Engineering

## Setup

Before we can start having fun, let's setup the key libraries and functions we'll need. These will allow us to:
- Call OpenAI's API
- Call Bing's API

In [ ]:
%pip install python-dotenv
%pip install pprintpp
%pip install openai

## Environment Variables

Create an `.env` file in the root directory of this project. It should contain the following variables:

```env
BING_SEARCH_V7_SUBSCRIPTION_KEY="<your key here>"
BING_SEARCH_V7_ENDPOINT="https://api.bing.microsoft.com/"
OPENAI_KEY="<your openai key>"
OPENAI_AZURE_KEY="<your azure openai key>"
OPENAI_AZURE_BASE_URL="<your azure openai endpoint>"
OPENAI_AZURE_API_VERSION="2023-05-15"
OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME="<your azure openai deployment name for gpt 3.5 or gpt 4>"
```

In [12]:
# Load environment variables
import os
from pprint import pprint
import requests
from dotenv import load_dotenv
load_dotenv()

True

## Create helper functions
These are helper functions that will allow us to call the APIs and get the results we need.


1. Search for content using Bing Search using `bing_search`

In [13]:
# This function returns search results via Bing Search API
def bing_search(query):
    # Add your Bing Search V7 subscription key and endpoint to your environment variables.
    subscription_key = os.environ["BING_SEARCH_V7_SUBSCRIPTION_KEY"]    
    endpoint = os.environ["BING_SEARCH_V7_ENDPOINT"] 

    # Ensure that the subscription key & endpoint have been set, else throw an error.
    if not subscription_key or not endpoint:
        raise Exception("Set your environment variables for your subscription key and endpoint.")

    # Add a trailing slash to the endpoint if needed
    if endpoint[-1] != "/":
        endpoint = endpoint + "/"
    endpoint = endpoint + "v7.0/search"


    # Construct a request
    mkt = "en-US"
    params = { "q": query, "mkt": mkt }
    headers = { "Ocp-Apim-Subscription-Key": subscription_key }

    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()

        search_results = response.json()
    except Exception as ex:
        raise ex
    
    return search_results

In [14]:
# Perform a test search for the word "Microsoft"
result = bing_search("Microsoft")
# Print the first 2 results within the JSON response.
pprint(result["webPages"]["value"][0:2], indent=4, width=100, depth=2)

[   {   'dateLastCrawled': '2023-06-26T02:04:00.0000000Z',
        'deepLinks': [...],
        'displayUrl': 'https://www.microsoft.com/en-us',
        'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
        'isFamilyFriendly': True,
        'isNavigational': True,
        'language': 'en',
        'name': 'Microsoft – Cloud, Computers, Apps & Gaming',
        'snippet': 'Explore Microsoft products and services for your home or business. Shop '
                   'Surface, Microsoft 365, Xbox, Windows, Azure, and more. Find downloads and get '
                   'support.',
        'url': 'https://www.microsoft.com/en-us/'},
    {   'dateLastCrawled': '2023-06-25T15:14:00.0000000Z',
        'displayUrl': 'https://support.microsoft.com',
        'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.1',
        'isFamilyFriendly': True,
        'isNavigational': False,
        'language': 'en',
        'name': 'Microsoft Support',
        'searchTags': [...],
        'snippet

2. Pretty Prining functions:

    - `bold` - Print text in bold
    - `wrap` - Wraps text with a default width of 100 characters.

In [66]:
import textwrap

def bold(text):
    print(f'\033[1m{text}\033[0m')

# Testing the bold function
bold("Testing the bold functionality.")

def italic(text):
    print(f'\033[3m{text}\033[0m')

# Testing the italic function
italic("Testing the italic functionality.")

def wrap(text, line_length=100):
    for line in text.split('\n'):
        wrapper = textwrap.TextWrapper(width=line_length)
        word_list = wrapper.wrap(text=line)

        # Print each line
        for l in word_list:
            print(l)

# Testing the wrap function
wrap("The quick brown fox jumped over the lazy dog.", 10)

def box(text):    
    bold(f'\n\n┌{"─" * len(text)}┐')
    bold(f'│{text}│')
    bold(f'└{"─" * len(text)}┘')

# Testing the box function
box("Heading")

Testing the bold functionality.
Testing the italic functionality.
The quick
brown fox
jumped
over the
lazy dog.


┌───────┐
│Heading│
└───────┘


3. Message helper functions: OpenAI's Chat Completions expects messages in the following format:
    ```python
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"}
    ]
    ```
    To assist with this, we have the following functions:
    
    - `validate_message` - Validates that a message is in the expected format.
    - `print_messages` - Prints messages.

In [16]:
def validate_messages(messages):
    # Define the acceptable roles
    valid_roles = ["system", "user", "assistant"]

    # Check if messages is a list
    if not isinstance(messages, list):
        raise TypeError("Input should be a list")

    # Iterate over the list of messages
    for message in messages:
        # Check if message is a dictionary
        if not isinstance(message, dict):
            raise TypeError("Each message should be a dictionary")

        # Check if the necessary keys exist in the dictionary
        if "role" not in message or "content" not in message:
            raise ValueError("Each message should have 'role' and 'content' keys")

        # Check if role is valid
        if message["role"] not in valid_roles:
            raise ValueError(f"Role should be one of {valid_roles}")

    # If all messages pass the checks, return True
    return True

# Test valid messages
if validate_messages([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"}
]):
    print("Messages are valid!")

# Test invalid messages
try:
    validate_messages([
        "You are a helpful assistant.",
    ])
except Exception as ex:    
    print("Messages are invalid because:")
    print(ex)

def print_messages(messages):
    # Validate the messages first
    if validate_messages(messages):
        # Iterate over the messages and print each one in the desired format
        for message in messages:                        
            bold(f'\n[{message["role"]}]:')
            wrap(message["content"])

# Test the print_messages function
print_messages([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"}
])

Messages are valid!
Messages are invalid because:
Each message should be a dictionary

[system]:
You are a helpful assistant.

[user]:
Knock knock.

[assistant]:
Who's there?


4. Helper message to call the ChatCompletions endpoint (direct or via azure) using `perform_chat_completion`. 
    ***Note:*** Can call azure OR openai directly based on the type parameter. Defaults to openai.

    This function has the following parameters:

    | Parameter | Description | Is Required | Default Value |
    | --- | --- | --- | --- |
    | `messages` | The messages to send to the API. | Yes | N/A |
    | `type` | The type of API to call. Can be `openai` or `azure`. | No | `openai` |
    | `temperature` | What temperature to use for the API call. A value between 0 and 2 | No | `0` |
    | `top_p` | What top_p to use for the API call. A value between 0 and 1 | No | `1` |
    | `max_tokens` | What max_tokens to use for the API call. A value between 1 and 4096 | No | `2048` |

In [17]:
import sys

# This function calls the OpenAI (or Azure OpenAI)'s completion endpoint
def perform_chat_completion(messages, type="openai", temperature=0, top_p=0.5, max_tokens=2048):
    # perform checks else raise exception
    if type != 'openai' and type != 'azure':
        raise Exception("type must be either openai or azure")
    
    validate_messages(messages)
    
    if temperature < 0 or temperature > 2:
        raise Exception("temperature must be between 0 and 2")
    
    if top_p < 0 or top_p > 1:
        raise Exception("top_p must be between 0 and 1")

    if max_tokens < 1 or max_tokens > 4096:
        raise Exception("max_tokens must be between 1 and 2048")


    # check if type is openai or azure
    if type == "openai":        
        if not os.environ["OPENAI_KEY"]:
            raise Exception("Set your OPENAI_KEY environment variable.")
        
        import openai        
        openai.api_type = "openai"
        openai.api_version = None
        openai.api_base = "https://api.openai.com/v1/"
        openai.api_key = os.environ["OPENAI_KEY"]
            

        try:
            results = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens
            )
        except Exception as ex:
            raise ex
    else:
        if not os.environ["OPENAI_AZURE_KEY"]:
            raise Exception("Set your OPENAI_AZURE_KEY environment variable.")
        if not os.environ["OPENAI_AZURE_BASE_URL"]:
            raise Exception("Set your OPENAI_AZURE_BASE_URL environment variable.")
        if not os.environ["OPENAI_AZURE_API_VERSION"]:
            raise Exception("Set your OPENAI_AZURE_API_VERSION environment variable.")
        if not os.environ["OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME"]:
            raise Exception("Set your OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME environment variable.")
                
        import openai
        openai.api_type = 'azure'
        openai.api_version = os.environ["OPENAI_AZURE_API_VERSION"]
        openai.api_base = os.environ["OPENAI_AZURE_BASE_URL"]
        openai.api_key = os.environ["OPENAI_AZURE_KEY"]
        
        try:
            results = openai.ChatCompletion.create(
                deployment_id=os.environ["OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME"],
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens
            )
        except Exception as ex:
            raise ex
        
    return results        


In [18]:
# Note: You can skip this step if you are using Azure OpenAI Service
box("Call Chat Completions endoint using OpenAI")
completion_message_request = [
   {"role": "system", "content": "You are a helpful assistant."},
   {"role": "user", "content": "Hi there."}
]
print_messages(completion_message_request)
chat_completion = perform_chat_completion(completion_message_request, "openai")
print_messages([chat_completion["choices"][0]["message"]])






┌──────────────────────────────────────────┐
│Call Chat Completions endoint using OpenAI│
└──────────────────────────────────────────┘

[system]:
You are a helpful assistant.

[user]:
Hi there.

[assistant]:
Hello! How can I assist you today?


In [19]:
# Note: You can skip this step if you are using OpenAI's API
box("Call Chat Completions endoint using  Azure")
completion_message_request = [
   {"role": "system", "content": "You are a helpful assistant."},
   {"role": "user", "content": "Hi there."}
]
print_messages(completion_message_request)
chat_completion = perform_chat_completion(completion_message_request, "azure")
print_messages([chat_completion["choices"][0]["message"]])



┌──────────────────────────────────────────┐
│Call Chat Completions endoint using  Azure│
└──────────────────────────────────────────┘

[system]:
You are a helpful assistant.

[user]:
Hi there.

[assistant]:
Hello! How can I assist you today?


## Prompt Engineering 

### Example 1: Simple conversational scenarios

The following code snippet shows the most basic way to use the ChatGPT and GPT-4 models with the Chat Completion API. 

In this example we ask the model to complete the following conversation:
```log
system: You are a helpful assistant.
user: Who were the founders of Microsoft?
assistant:
```

In [70]:
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who were the founders of Microsoft?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
box("Scenario 1: Simple conversational scenarios")
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────────────────┐
│Scenario 1: Simple conversational scenarios│
└───────────────────────────────────────────┘

[system]:
You are a helpful assistant.

[user]:
Who were the founders of Microsoft?

[assistant]:
The founders of Microsoft are Bill Gates and Paul Allen.


As you can see in the code above, the chat completion API expects messages to be in the following format:   
```json
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```

Depending on the role you should use the following values:
| Role | Purpose | Explaination | Example |
| --- | --- | --- | --- |
| `system` | Provide some context and/or instructions to the model. | The system message is included at the beginning of the prompt and is used to prime the model and you can include a variety of information in the system message including: <br> - A brief description of the assistant <br> - The personality of the assistant <br> - Instructions for the assistant <br> - Data or information needed for the model | `{"role": "system", "content": "You are a helpful assistant."}` |
| `user` | A question/message for the model to actually respond to. | After the system message, you can include a series of messages between the user and the assistant. You denote who the message is from by setting the role to user or assistant. | `{"role": "user", "content": "Knock knock."}` |



**⚠️ You might be wondering how this is different from the previous models.**

Unlike previous GPT-3 models, the ChatGPT model is specifically designed to be a conversational interface. The conversational nature of the model makes it easier to interact with and to take advantage of the full power of its capabilities.

Previous models were text-in and text-out (i.e., they accepted a prompt string and returned a completion to append to the prompt).

The ChatGPT model is conversation-in and message-out. (i.e., it expects a prompt string that is formatted in a specific chat-like transcript format and returns a completion that represents a model-written message in the chat)


### Example 2: Non conversational scenarios

While the Chat Completion API is optimized to work with multi-turn conversations, it also can be used for non chat scenarios. 

This example is set as follows:
| Role | Explanation |
| --- | --- | 
| `system` | You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand. | 
| `user` | Thinking, Fast and Slow by Daniel Kahneman |



In [71]:
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [
    {"role": "system", "content": "You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand."},
    {"role": "user", "content": "Thinking, Fast and Slow by Daniel Kahneman"}
]
completion_response = perform_chat_completion(prompt_messages, type)
box("Scenario 2: Non conversational scenarios")
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌────────────────────────────────────────┐
│Scenario 2: Non conversational scenarios│
└────────────────────────────────────────┘

[system]:
You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a
book and you will respond with a short summary. Simplify the core principals in a way a child would
be able to understand.

[user]:
Thinking, Fast and Slow by Daniel Kahneman

[assistant]:
"Thinking, Fast and Slow" is a book about how our brains work. It explains that we have two
different ways of thinking: fast and slow. Fast thinking is when we make quick decisions without
really thinking about them. Slow thinking is when we take our time and really think things through.
The book also talks about how our brains can sometimes trick us into making bad decisions, and how
we can learn to make better decisions by being aware of these tricks.


# Example 3: Role Prompting or Personas

Here lets explore the `Be specific` and `Be descriptive` best practices listed at https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/prompt-engineering#best-practices.

- `Be Specific`: Leave as little to interpretation as possible. Restrict the operational space.
- `Be Descriptive`: Use analogies.

In this example we will show how being specific and descriptive can help us get better results. 

**Scenario 3.1: Basic prompt**

| Role | Explanation |
| --- | --- |
| `user` | What is 1000*1000/20*50? |

**Scenario 3.2: Better prompt using Role Prompting**

| Role | Explanation |
| --- | --- |
| `system` | Assume you are a math teacher and you are going to teach Order Of Operations Activities (BODMAS, BIMDAS, PEMDAS, GEMS & More) to your students. |
| `user` | What is 1000*1000/20*50? |


In [35]:
box("Scenario 3.1: Basic prompt")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [    
    {"role": "user", "content": "What is 1000*1000/20*50?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌──────────────────────────┐
│Scenario 3.1: Basic prompt│
└──────────────────────────┘

[user]:
What is 1000*1000/20*50?

[assistant]:
1000*1000/20*50 = 50,000


In [37]:
box("Scenario 3.2: Better prompt using Role Prompting")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [    
    {"role": "user", "content": "Assume you are a math teacher and you are going to teach PEMDAS (Parentheses, Exponents, Multiplication and Division from left to right, and Addition and Subtraction from left to right) to your students. What is 1000*1000/20*50?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌────────────────────────────────────────────────┐
│Scenario 3.2: Better prompt using Role Prompting│
└────────────────────────────────────────────────┘

[user]:
Assume you are a math teacher and you are going to teach PEMDAS (Parentheses, Exponents,
Multiplication and Division from left to right, and Addition and Subtraction from left to right) to
your students. What is 1000*1000/20*50?

[assistant]:
To solve this problem using PEMDAS, we need to follow the order of operations:
1. Parentheses (none)
2. Exponents (none)
3. Multiplication and Division from left to right:
1000 * 1000 = 1,000,000
1,000,000 / 20 = 50,000
50,000 * 50 = 2,500,000
4. Addition and Subtraction from left to right (none)
Therefore, 1000*1000/20*50 = 2,500,000.


### Example 4: Providing instructions to get expected results

To ensure the model provides the expected results, you can provide instructions to the model within the system message.

In this example, show the difference beteen providing instructions and not providing instructions. We will use the same prompt for both examples, but the system message will be different.

**Scenario 4.1: No instructions**

| Role | Explanation |
| --- | --- |
| `system` | You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand. |
| `user` | How to build your Technology Strategy by Rohit Lakhanpal |

**Scenario 4.2: With instructions**

| Role | Explanation |
| --- | --- |
| `system` | You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand. <br> <br> **Instructions:** <br> - If you don't know about the book, say "I don't know". <br> - Don't make up information. |
| `user` | How to build your Technology Strategy by Rohit Lakhanpal |


**Note**: No book called "How to build your Technology Strategy" by "Rohit Lakhanpal" exists. The model will make up information if you don't provide instructions. 

In [39]:
box("Scenario 4.1: No instructions")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
system_message = "You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand."
book = "How to build your Technology Strategy by Rohit Lakhanpal"
prompt_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": book}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌─────────────────────────────┐
│Scenario 4.1: No instructions│
└─────────────────────────────┘

[system]:
You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a
book and you will respond with a short summary. Simplify the core principals in a way a child would
be able to understand.

[user]:
How to build your Technology Strategy by Rohit Lakhanpal

[assistant]:
This book is about making a plan for using technology in a smart way. It helps you think about what
you want to achieve with technology and how to make it happen. It's like making a map to get to your
destination, but for technology!


In [38]:
box("Scenario 4.2: With instructions")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
system_message = """You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand.

#Instructions:
- If you don't know about the book, say "I don't know". 
- Don't make up information.
"""
prompt_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "How to be a Tech Stragist by Rohit Lakhanpal"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────┐
│Scenario 4.2: With instructions│
└───────────────────────────────┘

[system]:
You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a
book and you will respond with a short summary. Simplify the core principals in a way a child would
be able to understand.
#Instructions:
- If you don't know about the book, say "I don't know".
- Don't make up information.

[user]:
How to be a Tech Stragist by Rohit Lakhanpal

[assistant]:
I don't know.


As seen in example above, when you don't provide instructions, the model will make up information. This is known as `hallucination`. To combat this, we used the system message to provide instructions to the model. This is part of the best practice known as `Giving the model an out`.

**Give the model an out**: It can sometimes be helpful to give the model an alternative path if it is unable to complete the assigned task. For example, when asking a question over a piece of text you might include something like `respond with 'not found' if the answer is not present`. This can help the model avoid generating false responses.

Learn more at https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/prompt-engineering#best-practices

### Example 5: Zero Shot, One Shot, Few Shot

In this example we will show how to use the `Zero Shot`, `One Shot` and `Few Shot` capabilities of the Chat Completion API. 

**Scenario 5.1: Zero Shot**

Only the command and the unsolved input is provided to the model.

| Role | Explanation |
| --- | --- |
| `system` | Suggest three names for an animal that is a superhero. |
| `user` | Dog |


**Scenario 5.2: One Shot**

One example is provided to the model before an unsolved problem is asked.

| Role | Explanation |
| --- | --- |
| `system` | Suggest three names for an animal that is a superhero. |
| `user` | Cat |
| `assistant` | Cat Damon, Pawsley Snipes and Audrey Hepurrn |
| `user` | Dog |


**Scenario 5.3: Few Shot**

Few solved examples are provided to the model before an unsolved problem is asked.

| Role | Explanation |
| --- | --- |
| `system` | Suggest three names for an animal that is a superhero. |
| `user` | Cat |
| `assistant` | Cat Damon, Pawsley Snipes and Audrey Hepurrn |
| `user` | Horse |
| `assistant` | Neigh-talie Portman, Gallop Gadot and David Hasselhoof |
| `user` | Dog |


In [54]:
box("Scenario 5.1: Generate Dog named using Zero Shot")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌────────────────────────────────────────────────┐
│Scenario 5.1: Generate Dog named using Zero Shot│
└────────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Dog

[assistant]:
1. Superpup
2. Mighty Mutt
3. Captain Canine


In [55]:
box("Scenario 5.2: Generate Dog named using One Shot")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────────────────────┐
│Scenario 5.2: Generate Dog named using One Shot│
└───────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Dog

[assistant]:
Bark Kent, Captain Canine and Wonder Woof


In [57]:
box("Scenario 5.3: Generate Dog named using Few Shot")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},    
    {"role": "user", "content": "Horse"},
    {"role": "assistant", "content": "Neigh-talie Portman, Gallop Gadot and David Hasselhoof"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────────────────────┐
│Scenario 5.3: Generate Dog named using Few Shot│
└───────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Horse

[assistant]:
Neigh-talie Portman, Gallop Gadot and David Hasselhoof

[user]:
Dog

[assistant]:
Bark Kent, Wonder Pug and Captain Canine


### Example 6: Exploring API parameters

The Chat Completion API has a number of parameters that can be used to control the output of the model. 

In this example we will explore the following parameters:

- `temperature`: 

    Controls the randomness of the model. Lower values make the model more deterministic and higher values make the model more random.
    At zero the model becomes deterministic (default: 0.3 – 0.7). Lower values are better for "accuracy", higher values better for "creativity".

- `top_p`:

    Controls the diversity via nucleus sampling. When set to 0.5 means half of all likelihood-weighted options are considered (default: 1). Can act as a filter for unlikely completions.

Let's explore the results we get when we make the model more deterministic v/s more random (or creative). We will use the same prompt for both examples based on the last example.


In [68]:
box("Scenario 6.1: Generate Dog names more deterministically")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type, temperature=0, top_p=0)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])
italic("Likely you'll get the same response most of the time.")



┌───────────────────────────────────────────────────────┐
│Scenario 6.1: Generate Dog names more deterministically│
└───────────────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Dog

[assistant]:
Bark Kent, Captain Canine and Wonder Woof
Likely you'll get the same response most of the time.


In [69]:
box("Scenario 6.2: Generate Dog names more randomly or creatively")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type, temperature=2, top_p=1)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])
italic("More likely you'll get a different response each time.")



┌────────────────────────────────────────────────────────────┐
│Scenario 6.2: Generate Dog names more randomly or creatively│
└────────────────────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Dog

[assistant]:
Ace Barkman, Blu Thunder Hounderson, Fido ElectroDasher
More likely you'll get a different response each time.


### Example 7: Specify output formats

You can specify output formats using the system message or in some cases, it can be inferred via the one-shot or few-shot examples.

In [72]:
box("Scenario 7: Specify output formats")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
system_message = """
You are an assistant designed to extract entities from text. Users will paste in a string of text and you will respond with entities you've extracted from the text as a JSON object. Here's an example of your output format:
{
   "name": "",
   "company": "",
   "phone_number": ""
}
"""
prompt_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Hello. My name is Robert Smith. I’m calling from Contoso Insurance, Delaware. My colleague mentioned that you are interested in learning about our comprehensive benefits policy. Could you give me a call back at (555) 346-9322 when you get a chance so we can go over the benefits?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌──────────────────────────────────┐
│Scenario 7: Specify output formats│
└──────────────────────────────────┘

[system]:
You are an assistant designed to extract entities from text. Users will paste in a string of text
and you will respond with entities you've extracted from the text as a JSON object. Here's an
example of your output format:
{
   "name": "",
   "company": "",
   "phone_number": ""
}

[user]:
Hello. My name is Robert Smith. I’m calling from Contoso Insurance, Delaware. My colleague mentioned
that you are interested in learning about our comprehensive benefits policy. Could you give me a
call back at (555) 346-9322 when you get a chance so we can go over the benefits?

[assistant]:
{
   "name": "Robert Smith",
   "company": "Contoso Insurance",
   "phone_number": "(555) 346-9322"
}


# Example 8: Chain of thought reasoning

Another powerful technique for improving the reliability of answers is to prompt the model to gradually reason out the answer rather than jumping immediately to the final answer. By 'thinking aloud' the model can be far more likely to arrive at the correct answer.

This technique called Chain-of-Thought, actually only discovered recently , is a super powerful technique, not only can it be used to provide model explainability (where sometimes GPT-3 is seen as a blackbox) but it can help the model reason and arrive at a desired output by simply just telling the model to think step by step. 


In [110]:
box("Scenario 8.1: Without Chain of thought reasoning")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
query = """Brain Teaser: 
What is the annual water demand of a single-family household containing four people at home an average of 200 days per year and using an average of 100 litres of water daily?
"""
prompt_messages = [
    {"role": "user", "content": query}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌────────────────────────────────────────────────┐
│Scenario 8.1: Without Chain of thought reasoning│
└────────────────────────────────────────────────┘

[user]:
Brain Teaser:
What is the annual water demand of a single-family household containing four people at home an
average of 200 days per year and using an average of 100 litres of water daily?

[assistant]:
The annual water demand would be 8,000 litres (100 litres per day x 4 people x 200 days).


In [111]:
box("Scenario 8.2: With Chain of thought reasoning")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
query = """Brain Teaser: 
What is the annual water demand of a single-family household containing four people at home an average of 200 days per year and using an average of 100 litres of water daily?

Let's think step by step and explain the calculation step by step.
"""
prompt_messages = [
    {"role": "user", "content": query}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌─────────────────────────────────────────────┐
│Scenario 8.2: With Chain of thought reasoning│
└─────────────────────────────────────────────┘

[user]:
Brain Teaser:
What is the annual water demand of a single-family household containing four people at home an
average of 200 days per year and using an average of 100 litres of water daily?
Let's think step by step and explain the calculation step by step.

[assistant]:
Step 1: Calculate the total number of days the household uses water in a year.
The household is at home for 200 days per year, so the total number of days they use water is 200.
Step 2: Calculate the total amount of water used by the household in a year.
The household uses an average of 100 litres of water daily, so the total amount of water used in a
year is:
100 litres/day x 200 days = 20,000 litres
Step 3: Divide the total amount of water used by the number of people in the household.
There are four people in the household, so the annual water demand per person is:


### Example 9: Prompt Chaining

Prompt chaining is a technique where you can chain multiple prompts together to get a desired output. Here we will modify the previous prompt and add a new prompt at the end to get a desired output.


In [112]:
box("Scenario 9: Prompt Chaining")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
query = """Brain Teaser: 
What is the annual water demand of a single-family household containing four people at home an average of 200 days per year and using an average of 100 litres of water daily?

Let's think step by step and explain the calculation step by step.
"""
response = """
Step 1: Calculate the total number of days the household uses water in a year.
The household is at home for 200 days per year, so the total number of days they use water is 200.
Step 2: Calculate the total amount of water used by the household in a year.
The household uses an average of 100 litres of water daily, so the total amount of water used in a
year is:
100 litres/day x 200 days = 20,000 litres
Step 3: Divide the total amount of water used by the number of people in the household.
The household contains four people, so the annual water demand per person is:
20,000 litres / 4 people = 5,000 litres/person
Therefore, the annual water demand of a single-family household containing four people at home an
average of 200 days per year and using an average of 100 litres of water daily is 20,000 litres, or
5,000 litres per person.
"""
prompt_messages = [
    {"role": "user", "content": query},
    {"role": "assistant", "content": response},
    {"role": "user", "content": "Therefore, the answer (number) is:"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────┐
│Scenario 9: Prompt Chaining│
└───────────────────────────┘

[user]:
Brain Teaser:
What is the annual water demand of a single-family household containing four people at home an
average of 200 days per year and using an average of 100 litres of water daily?
Let's think step by step and explain the calculation step by step.

[assistant]:
Step 1: Calculate the total number of days the household uses water in a year.
The household is at home for 200 days per year, so the total number of days they use water is 200.
Step 2: Calculate the total amount of water used by the household in a year.
The household uses an average of 100 litres of water daily, so the total amount of water used in a
year is:
100 litres/day x 200 days = 20,000 litres
Step 3: Divide the total amount of water used by the number of people in the household.
The household contains four people, so the annual water demand per person is:
20,000 litres / 4 people = 5,000 litres/person
Therefore, 

### Example 10: Provide grounding context via Retrieval Augmented Generation

If your use case relies on up-to-date, reliable information and is not purely a creative scenarios then one the most effective ways to provide reliable answers is to give the model data to draw its responses from grounding data.

In [136]:
box("Scenario 10: Grounding and Retrieval Augmented Generation")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
system_message = """You are an AI assistant that answers technical questions about Azure based on the context provided.

## Instructions
- You should always leverage the retrieved documents when the user is seeking information or whenever retrieved documents could be potentially helpful, regardless of your internal knowledge or information.
- You can leverage past responses and retrieved documents for generating relevant and interesting suggestions for the next user turn.
- You can only issue references to the documents as citation examples below. You should **never generate** URLs or links apart from the ones provided in retrieved documents.
- Retrieved documents may be incomplete or irrelevant. You don't make assumptions on the retrieved documents beyond strictly what's returned.
- If the retrieved documents do not contain sufficient information to answer user message completely, you can only include **facts from the retrieved documents** and do not add any information by itself.
- You can leverage information from multiple retrieved documents to respond **comprehensively**.
- You **should always** reference factual statements to the search results.
- If you are not sure, say "I don't know".
"""
user_shot_1 = """
What access does AOAI provide to OpenAI models?

```json
{
  "retrieved_documents": [
    {
      "[doc1]": {
        "title": "What is Azure OpenAI Service?",
        "content": "Azure OpenAI Service provides REST API access to OpenAI's powerful language models including the GPT-3, Codex and Embeddings model series. These models can be easily adapted to your specific task including but not limited to content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or our web-based interface in the Azure OpenAI Studio."
      }
    },
    {
      "[doc2]": {
        "title": "What is Azure Blob Storage?",
        "content": "Azure Blob Storage is Microsoft's object storage solution for the cloud. Blob Storage is optimized for storing massive amounts of unstructured data. Unstructured data is data that doesn't adhere to a particular data model or definition, such as text or binary data."
      }
    },
    {
      "[doc3]": {
        "title": "What is requirement for access Azure storage?",
        "content": "To access Azure Storage, you'll need an Azure subscription. If you don't already have a subscription, create a free account before you begin. All access to Azure Storage takes place through a storage account. For this quickstart, create a storage account using the Azure portal, Azure PowerShell, or Azure CLI. For help creating a storage account, see Create a storage account."
      }
    },
    {
      "[doc4]": {
        "title": "How to upload files to Azure Blob storage?",
        "content": "You can upload files and directories to Blob storage by using the AzCopy v10 command-line utility."
      }
    }
  ]
}
```
"""
assistant_shot_1 = """
Currently AOAI provides REST APIs, Python SDK, and web-based interface in the Azure OpenAI Studio to OpenAI's powerful language models [doc1].
"""
user_shot_2 = """
What is the deductible for the employee plan for a visit to Overlake in Bellevue?
```json
{
  "retrieved_documents": [
    {
      "[doc1]": {
        "title": "How to calculate the deductible?",
        "content": "Deductibles depend on whether you are in-network or out-of-network. In-network deductibles are $500 for employee and $1000 for family. Out-of-network deductibles are $1000 for employee and $2000 for family."
      }
    },
    {
      "[doc2]": {
        "title": "Is Overlake in-network?",
        "content": "Overlake is in-network for the employee plan."
      }
    },
    {
      "[doc3]": {
        "title": "What is Overlake?",
        "content": "Overlake is the name of the area that includes a park and ride near Bellevue."
      }
    },
    {
      "[doc4]": {
        "title": "What is the in-network list?",
        "content": "In-network institutions include Overlake, Swedish and others in the region"
      }
    }
  ]
}
```
"""
assistant_shot_2 = """
In-network deductibles are $500 for employee and $1000 for family [doc1]. Overlake is in-network for the employee plan [doc2][doc4].
"""
user_question = """
I want to programatically get a user's presence using Graph API. Can this be done using Application permissions type or does it need Delegated permissions?

```json
{
  "retrieved_documents": [
    {
      "[doc1]": {
        "title": "title: Tutorial - Web app accesses Microsoft Graph as the user | Azure",
        "content": "title: title: Tutorial - Web app accesses Microsoft Graph as the user | Azure\ndescription: In this tutorial, you learn how to access data in Microsoft Graph for a signed-in user.\nservices: microsoft-graph, app-service-web\nauthor: rwike77\nmanager: CelesteDG\ndocument_link: https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/includes/tutorial-connect-app-access-microsoft-graph-as-user/intro.md\ntitle: Tutorial - Web app accesses Microsoft Graph as the user | Azure\ndescription: In this tutorial, you learn how to access data in Microsoft Graph for a signed-in user.\nservices: microsoft-graph, app-service-web\nauthor: rwike77\nmanager: CelesteDG\nms.service: app-service\nms.topic: include\nms.workload: identity\nms.date: 11/02/2021\nms.author: ryanwi\nms.reviewer: stsoneff\nms.devlang: csharp, javascript\nms.custom: azureday1\nCustomer intent: As an application developer, I want to learn how to access data in Microsoft Graph for a signed-in user.\nms.subservice: web-apps\nLearn how to access Microsoft Graph from a web app running on Azure App Service.\n:::image type=\"content\" alt-text=\"Diagram that shows accessing Microsoft Graph.\" source=\"../../media/scenario-secure-app-access-microsoft-graph/web-app-access-graph.svg\" border=\"false\":::\nYou want to add access to Microsoft Graph from your web app and perform some action as the signed-in user. This section describes how to grant delegated permissions to the web app and get the signed-in user's profile information from Azure Active Directory (Azure AD).\nIn this tutorial, you learn how to:\n[!div class=\"checklist\"]\nGrant delegated permissions to a web app.\nCall Microsoft Graph from a web app for a signed-in user.\n[!INCLUDE quickstarts-free-trial-note]\nPrerequisites\nA web application running on Azure App Service that has the App Service authentication/authorization module enabled.\nGrant front-end access to call Microsoft Graph\nNow that you've enabled authentication and authorization on your web app, the web app is registered with the Microsoft identity platform and is backed by an Azure AD application. In this step, you give the web app permissions to access Microsoft Graph for the user. (Technically, you give the web app's Azure AD application the permissions to access the Microsoft Graph Azure AD application for the user.)\nIn the Azure portal menu, select Azure Active Directory or search for and select Azure Active Directory from any page.\nSelect App registrations > Owned applications > View all applications in this directory. Select your web app name, and then select API permissions.\nSelect Add a permission, and then select Microsoft APIs and Microsoft Graph.\nSelect Delegated permissions, and then select User.Read from the list. Select Add permissions.\nConfigure App Service to return a usable access token\nThe web app now has the required permissions to access Microsoft Graph as the signed-in user. In this step, you configure App Service authentication and authorization to give you a usable access token for accessing Microsoft Graph. For this step, you need to add the User.Read scope for the downstream service (Microsoft Graph): https://graph.microsoft.com/User.Read.\n[!IMPORTANT]\nIf you don't configure App Service to return a usable access token, you receive a CompactToken parsing failed with error code: 80049217 error when you call Microsoft Graph APIs in your code.\nAzure Resource Explorer\nGo to Azure Resource Explorer and using the resource tree, locate your web app. The resource URL should be similar to https://resources.azure.com/subscriptions/subscriptionId/resourceGroups/SecureWebApp/providers/Microsoft.Web/sites/SecureWebApp20200915115914.\nThe Azure Resource Explorer is now opened with your web app selected in the resource tree. At the top of the page, select Read/Write to enable editing of your Azure resources.\nIn the left browser, drill down to config > authsettingsV2.\nIn the authsettingsV2 view, select Edit. Find the login section of identityProviders -> azureActiveDirectory and add the following loginParameters settings: \"loginParameters\":[ \"response_type=code id_token\",\"scope=openid offline_access profile https://graph.microsoft.com/User.Read\" ]"
      }
    },
    {
      "[doc2]": {
        "title": "title: Tutorial - Web app accesses Microsoft Graph as the user | Azure",
        "content": "json\n\"identityProviders\": { \"azureActiveDirectory\": { \"enabled\": true, \"login\": { \"loginParameters\":[ \"response_type=code id_token\", \"scope=openid offline_access profile https://graph.microsoft.com/User.Read\" ] } } }\n},\nSave your settings by selecting PUT. This setting can take several minutes to take effect. Your web app is now configured to access Microsoft Graph with a proper access token. If you don't, Microsoft Graph returns an error saying that the format of the compact token is incorrect.\nAzure CLI\nUse the Azure CLI to call the App Service Web App REST APIs to get and update the auth configuration settings so your web app can call Microsoft Graph. Open a command window and login to Azure CLI:\nazurecli\naz login"
      }
    },
    {
      "[doc3]": {
        "title": "services: microsoft-graph, app-service-web",
        "content": "New-MgServicePrincipalAppRoleAssignment -ServicePrincipalId $managedIdentityObjectId -PrincipalId $managedIdentityObjectId -ResourceId $serverServicePrincipalObjectId -AppRoleId $appRoleId\n```\nAzure CLI\n```azurecli-interactive\naz login\nwebAppName=\"SecureWebApp-20201106120003\"\nspId=$(az resource list -n $webAppName --query [*].identity.principalId --out tsv)\ngraphResourceId=$(az ad sp list --display-name \"Microsoft Graph\" --query [0].id --out tsv)"
      }
    },
    {
      "[doc4]": {
        "title": "title: \"Quickstart: Configure an app to access a web API\"",
        "content": "title: title: \"Quickstart: Configure an app to access a web API\"\ndescription: In this quickstart, you configure an app registration representing a web API in the Microsoft identity platform to enable scoped resource access (permissions) to client applications.\nservices: active-directory\nauthor: cilwerner\nmanager: CelesteDG\nms.service: active-directory\nms.subservice: develop\nms.topic: quickstart\nms.workload: identity\nms.date: 05/05/2022\nms.author: cwerner\nms.custom: aaddev, contperf-fy21q1, mode-api\nms.reviewer: lenalepa, aragra, sureshja\ndocument_link: https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/active-directory/develop/quickstart-configure-app-access-web-apis.md\nIn addition to accessing your own web API on behalf of the signed-in user, your application might also need to access or modify the user's (or other) data stored in Microsoft Graph. Or you might have service or daemon app that needs to access Microsoft Graph as itself, performing operations without any user interaction.\nDelegated permission to Microsoft Graph\nConfigure delegated permission to Microsoft Graph to enable your client application to perform operations on behalf of the logged-in user, for example reading their email or modifying their profile. By default, users of your client app are asked when they sign in to consent to the delegated permissions you've configured for it.\nSign in to the Azure portal.\nIf you have access to multiple tenants, use the Directories + subscriptions filter :::image type=\"icon\" source=\"./media/quickstart-configure-app-access-web-apis/portal-01-directory-subscription-filter.png\" border=\"false\"::: in the top menu to select the tenant containing your client app's registration.\nSelect Azure Active Directory > App registrations, and then select your client application.\nSelect API permissions > Add a permission > Microsoft Graph\nSelect Delegated permissions. Microsoft Graph exposes many permissions, with the most commonly used shown at the top of the list.\nUnder Select permissions, select the following permissions:\n| Permission | Description |\n|--|--|\n| email | View users' email address |\n| offline_access | Maintain access to data you have given it access to |\n| openid | Sign users in |\n| profile | View users' basic profile |\n1. Select Add permissions to complete the process.\nWhenever you configure permissions, users of your app are asked at sign-in for their consent to allow your app to access the resource API on their behalf.\nAs an admin, you can also grant consent on behalf of all users so they're not prompted to do so. Admin consent is discussed later in the More on API permissions and admin consent section of this article.\nApplication permission to Microsoft Graph\nConfigure application permissions for an application that needs to authenticate as itself without user interaction or consent. Application permissions are typically used by background services or daemon apps that access an API in a \"headless\" manner, and by web APIs that access another (downstream) API.\nIn the following steps, you grant permission to Microsoft Graph's Files.Read.All permission as an example.\nSign in to the Azure portal.\nIf you have access to multiple tenants, use the Directories + subscriptions filter :::image type=\"icon\" source=\"./media/quickstart-configure-app-access-web-apis/portal-01-directory-subscription-filter.png\" border=\"false\"::: in the top menu to select the tenant containing your client app's registration.\nSelect Azure Active Directory > App registrations, and then select your client application.\nSelect API permissions > Add a permission > Microsoft Graph > Application permissions.\nAll permissions exposed by Microsoft Graph are shown under Select permissions.\nSelect the permission or permissions you want to grant your application. As an example, you might have a daemon app that scans files in your organization, alerting on a specific file type or name.\nUnder Select permissions, expand Files, and then select the Files.Read.All permission.\n1. Select Add permissions.\nSome permissions, like Microsoft Graph's Files.Read.All permission, require admin consent. You grant admin consent by selecting the Grant admin consent button, discussed later in the Admin consent button section.\nConfigure client credentials\nApps that use application permissions authenticate as themselves by using their own credentials, without requiring any user interaction. Before your application (or API) can access Microsoft Graph, your own web API, or another API by using application permissions, you must configure that client app's credentials"
      }
    },
    {
      "[doc5]": {
        "title": "title: Tutorial - .NET Web app accesses Microsoft Graph as the user | Azure",
        "content": "title: title: Tutorial - .NET Web app accesses Microsoft Graph as the user | Azure\ndescription: In this tutorial, you learn how to access data in Microsoft Graph for a signed-in user from a .NET web app.\nservices: microsoft-graph, app-service-web\nauthor: rwike77\nmanager: CelesteDG\ndocument_link: https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/scenario-secure-app-access-microsoft-graph-as-user.md\n\nAzureAd specifies the configuration for the Microsoft.Identity.Web library. In the Azure portal, select Azure Active Directory from the portal menu and then select App registrations. Select the app registration created when you enabled the App Service authentication/authorization module. (The app registration should have the same name as your web app.) You can find the tenant ID and client ID in the app registration overview page. The domain name can be found in the Azure AD overview page for your tenant.\nGraph specifies the Microsoft Graph endpoint and the initial scopes needed by the app.\n```json\n{ \"AzureAd\": { \"Instance\": \"https://login.microsoftonline.com/\", \"Domain\": \"fourthcoffeetest.onmicrosoft.com\", \"TenantId\": \"[tenant-id]\", \"ClientId\": \"[client-id]\", // To call an API \"ClientSecret\": \"[secret-from-portal]\", // Not required by this scenario \"CallbackPath\": \"/signin-oidc\" },\n\"Graph\": { \"BaseUrl\": \"https://graph.microsoft.com/v1.0\", \"Scopes\": \"user.read\" }, \"Logging\": { \"LogLevel\": { \"Default\": \"Information\", \"Microsoft\": \"Warning\", \"Microsoft.Hosting.Lifetime\": \"Information\" } }, \"AllowedHosts\": \"*\"\n}\n```\nCall Microsoft Graph on behalf of the user\nThe following example shows how to call Microsoft Graph as the signed-in user and get some user information. The GraphServiceClient object is injected into the controller, and authentication has been configured for you by the Microsoft.Identity.Web library.\n```csharp\n// Index.cshtml.cs\nusing System.Threading.Tasks;\nusing Microsoft.AspNetCore.Mvc.RazorPages;\nusing Microsoft.Graph;\nusing System.IO;\nusing Microsoft.Identity.Web;\nusing Microsoft.Extensions.Logging;\n// Some code omitted for brevity.\n[AuthorizeForScopes(Scopes = new[] { \"user.read\" })]\npublic class IndexModel : PageModel\n{ private readonly ILogger _logger; private readonly GraphServiceClient _graphServiceClient;\n"
      }
    }
  ]
}
```
"""
prompt_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_shot_1},
    {"role": "assistant", "content": assistant_shot_1},
    {"role": "user", "content": user_shot_2},
    {"role": "assistant", "content": assistant_shot_2},
    {"role": "user", "content": user_question},
]

completion_response = perform_chat_completion(prompt_messages, type)
#print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌─────────────────────────────────────────────────────────┐
│Scenario 10: Grounding and Retrieval Augmented Generation│
└─────────────────────────────────────────────────────────┘

[assistant]:
To get a user's presence using Graph API, you need to use Delegated permissions [doc1].


### TODO:
- Example X: Auto Validation
- Example X: Query based summarisation
- Example X: Edge cases and Responsible AI
- Example X: Prime the output

### Example X: Retrieval Augmented Generation

If your use case relies on up-to-date, reliable information and is not purely a creative scenarios then one the most effective ways to provide reliable answers is to give the model data to draw its responses from grounding data.